# Prediction Models

## Imports

In [7]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor


## Load Data

In [8]:
df_teams = pd.read_csv("dataset/cleaned/teams.csv")
df_teams_post = pd.read_csv("dataset/cleaned/teams_post.csv")
df_series_post = pd.read_csv("dataset/cleaned/series_post.csv")
df_players = pd.read_csv("dataset/cleaned/players.csv")
df_players_teams = pd.read_csv("dataset/cleaned/players_teams.csv")
df_coaches = pd.read_csv("dataset/cleaned/coaches.csv")
df_awards_players = pd.read_csv("dataset/cleaned/awards_players.csv")


## Models

In [9]:
# ### dps vemos estaticas de rebounds, steals, blocks, turnovers, etc para prever as estatisticas da equipa no ano seguinte

# save year 2010
df_teams_2010 = df_teams[df_teams.year == 10]
# 
# 
# Vai depender muito do que eles nos dão para testarmos.... provavelmente só nos vão dar a composição das equipas e não estatisiticas de jogos
# 
# 

#drop where year equals 2010
df_teams = df_teams[df_teams.year != 10]
tmids = df_teams_2010['tmID']
confIds = df_teams_2010['confID']
df_teams = df_teams.drop(columns=['tmID', 'year', 'confID'], axis=1)
df_teams_2010 = df_teams_2010.drop(columns=['tmID', 'year', 'confID'], axis=1)
# remove year and tmID

X, Y = df_teams.drop(columns=['winPercentage'], axis=1), df_teams['winPercentage']

best_rf = RandomForestRegressor(n_estimators = 600,min_samples_split = 2,min_samples_leaf = 4,max_features = 'auto',max_depth = 40,bootstrap = True)
base_rf = RandomForestRegressor()
best_model = best_rf.fit(X, Y)
base_model = base_rf.fit(X, Y)


X_TEST = df_teams_2010.drop(columns=['winPercentage'], axis=1)
Y_TEST = df_teams_2010['winPercentage']


def mae(test,pred): # Mean absolute error 
    err = np.mean(abs(test-pred))
    return err
y_pred_base = base_model.predict(X_TEST)
y_pred_best = best_model.predict(X_TEST)
print('Mean absolute error (base rf model): {} '.format(mae(Y_TEST,y_pred_base)))
print('Mean absolute error (best rf model): {} '.format(mae(Y_TEST,y_pred_best)))
print(y_pred_base)
print(y_pred_best)

#add tmid again to df_teams_2010
df_teams_2010['tmID'] = tmids




# for every objet show the predicted winPercentage and the actual winPercentage
i=0
for index, row in df_teams_2010.iterrows():
    print(f"Team: {row['tmID']}\tPredicted winPercentage: {y_pred_best[i]}\tActual winPercentage: {row['winPercentage']}")
    i+=1


/home/m/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Mean absolute error (base rf model): 0.023363076923076823 
Mean absolute error (best rf model): 0.022559392277514963 
[0.53007 0.44052 0.45693 0.54437 0.63437 0.54085 0.46408 0.40226 0.67342
 0.32326 0.50913 0.57822 0.50668]
[0.53177807 0.44890795 0.45590029 0.54208297 0.62615728 0.53601878
 0.46423849 0.38764722 0.66631172 0.32952742 0.50853118 0.57047601
 0.50725607]
Team: ATL	Predicted winPercentage: 0.5317780707163212	Actual winPercentage: 0.529
Team: CHI	Predicted winPercentage: 0.4489079500083248	Actual winPercentage: 0.471
Team: CON	Predicted winPercentage: 0.45590029378029395	Actual winPercentage: 0.471
Team: DET	Predicted winPercentage: 0.5420829673648867	Actual winPercentage: 0.529
Team: IND	Predicted winPercentage: 0.626157280992156	Actual winPercentage: 0.647
Team: LAS	Predicted winPercentage: 0.5360187803343243	Actual winPercentage: 0.529
Team: MIN	Predicted winPercentage: 0.46423848784371985	Actual winPercentage: 0.412
Team: NYL	Predicted winPercentage: 0.3876472178574425

In [10]:
#
#
# NOTE: This is tested against year 2009. It is able to predict the exact teams that made it to the playoffs because the dataset contains the playoff statistics features for that year.
#      However, the teacher won't probably give us the playoff statistics for the year 2010, so we will have to predict the teams that will make it to the playoffs based on other features.
#
#
#
# Make a prediction for the 2010 season
# say which teams will make it to the playoffs ... only four teams from each conference make it to the playoffs

df_teams_2010['confID'] = confIds

# add predWinPercentage to df_teams_2010
df_teams_2010['predWinPercentage'] = y_pred_best

ea_conf = df_teams_2010[df_teams_2010.confID == 'EA']
we_conf = df_teams_2010[df_teams_2010.confID == 'WE']


# get the top 4 teams from each conference
ea_conf = ea_conf.sort_values(by=['predWinPercentage'], ascending=False)
we_conf = we_conf.sort_values(by=['predWinPercentage'], ascending=False)


# print the teams that will make it to the playoffs

print("Eastern Conference")
for index, row in ea_conf.head(4).iterrows():
    print(f"{row['tmID']} with probality of {row['predWinPercentage']}")
print("Western Conference")
for index, row in we_conf.head(4).iterrows():
    print(f"{row['tmID']} with probality of {row['predWinPercentage']}")



Eastern Conference
IND with probality of 0.626157280992156
DET with probality of 0.5420829673648867
ATL with probality of 0.5317780707163212
WAS with probality of 0.5072560734298598
Western Conference
PHO with probality of 0.6663117214405967
SEA with probality of 0.5704760090696345
LAS with probality of 0.5360187803343243
SAS with probality of 0.5085311798581534
